In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn import mixture
import random
import time
from datetime import datetime

from pprint import *
from smrt_util import *
# import cPickle as Pickle
import _pickle as Pickle

/opt/conda/lib/python3.6/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
# where the data is
tx_filename = 'tx_data/ezlink-transaction.csv'
msg_filename = 'tx_data/all.msg'

In [63]:
# Step 1. Preprocessing the transaction data
preprocessing(tx_filename,msg_filename)

[Info] Preprocessing start.... (May take long time)
*. Step 1. Load csv file
Done! 20.626622200012207
*. Step 2. Concate O-D
   - 2.1. transform internal id to node id
   --> Done! 0.8830456733703613
   - 2.2. Concate o and d
   --> Done! 134.28537583351135
Done! 135.16895627975464
*. Step 3. Transfer date time into time stamp
   - 3.1. Compute entry timestamp
   -->  372.8730516433716
   - 3.2. Compute depart timestamp
   -->  114.91920685768127
Done! 487.7929606437683
*. Step 4. Save DataFrame
Done! 18.274646043777466


,card_id,ori_id,dest_id,entry_date,entry_time,duration,od,entry_ts,depart_ts,depart_date,depart_time,entry_ticks,depart_ticks
0,43DA8F714274DF099CCB1551E8F42C46DA1051E8,80,59,2016-05-18,16:13:59,11.367,80_59,1.463588e+09,1.463589e+09,2016-05-18,16:25:21,58439,59121
1,6F912C3E53DBE8D0C9DFDF5803DF84BEF7899186,80,50,2016-05-18,17:21:28,37.950,80_50,1.463592e+09,1.463594e+09,2016-05-18,17:59:25,62488,64765
2,794F3B3782B841414178964724ED1EE7D4F0CAF9,80,86,2016-05-18,18:42:51,21.283,80_86,1.463597e+09,1.463598e+09,2016-05-18,19:04:07,67371,68647
3,D7451C4AA5DA6A3F755884B253DAADEDFBF75EA1,80,51,2016-05-18,22:27:35,59.133,80_51,1.463610e+09,1.463614e+09,2016-05-18,23:26:42,80855,84402
4,9D35DA9AE0299EFFC1DD13AEB8B24D0BCA275360,80,26,2016-05-19,06:44:45,33.733,80_26,1.463640e+09,1.463642e+09,2016-05-19,07:18:28,24285,26308
5,7C9BF2CC2753891017AA04FDD660487471ED0849,80,72,2016-05-19,07:28:56,18.483,80_72,1.463643e+09,1.463644e+09,2016-05-19,07:47:24,26936,28044
6,6AFCA9B39DB3244D561D6DEBFBDE1DBA028DEBE3,80,88,2016-05-19,07:30:53,24.417,80_88,1.463643e+09,1.463645e+09,2016-05-19,07:55:18,27053,28518
7,557C58AD01DF4B10CDE1A449352A898C8986147E,80,44,2016-05-19,18:08:53,40.883,80_44,1.463681e+09,1.463684e+09,2016-05-19,18:49:45,65333,67785
8,8B37EBEE34DD8E932895C53BD89F12246CBFCF31,80,26,2016-05-19,16:37:01,34.233,80_26,1.463676e+09,1.463678e+09,2016-05-19,17:11:14,59821,61874
9,2A2D169D8EBAAF5C18B440272A7D7E8BB142AF56,80,59,2016-05-19,18:15:05,11.750,80_59,1.463682e+09,1.463682e+09,2016-05-19,18:26:50,65705,66410


In [3]:
# Load dataframe
df = df = load_df(msg_filename)

In [4]:
# Filtering
# df = get_df_by_od(df,o,d)
df = get_df_by_date(df,"2016-05-16","2016-05-16")
#df = get_df_by_time(df,"14:00:00","15:00:00",mode='depart')

In [6]:
df.head()

,card_id,ori_id,dest_id,entry_date,entry_time,duration,od,entry_ts,depart_ts,depart_date,depart_time,entry_ticks,depart_ticks
79,07B3F084C6CFB9A31C294D85001D385EBD64D401,54,64,2016-05-16,06:37:36,27.283,54_64,1.463381e+09,1.463382e+09,2016-05-16,07:04:52,23856,25492
80,07B3F084C6CFB9A31C294D85001D385EBD64D401,64,54,2016-05-16,15:22:14,24.800,64_54,1.463412e+09,1.463414e+09,2016-05-16,15:47:02,55334,56822
81,58E5F5B5549670256BB9EAE384F4676941D64600,54,78,2016-05-16,07:21:04,31.950,54_78,1.463383e+09,1.463385e+09,2016-05-16,07:53:01,26464,28381
82,02A82D3FDDDE2C13771FF7A565AEB9A922D21C92,54,85,2016-05-16,07:25:35,49.567,54_85,1.463384e+09,1.463387e+09,2016-05-16,08:15:09,26735,29709
83,E862CBE7DEC8DB7CFAC9F4C3F7FD2E983A9CC884,54,67,2016-05-16,08:10:00,37.333,54_67,1.463386e+09,1.463388e+09,2016-05-16,08:47:19,29400,31639


In [5]:
ticks_to_slice(17615)

2

In [6]:
# compute slices for each entry time and depart time for further use
df['entry_slice']=df['entry_ticks'].apply(lambda x:ticks_to_slice(x))
df['depart_slice']=df['depart_ticks'].apply(lambda x:ticks_to_slice(x))

In [8]:
# load get_od_ksp paths
# get od_ksp dict.... key:o_d --> value: [time,path_id,[sp],dict:o_d-->time]
# path_id: o_d_i (i: the i-th shortest path from o to d)
od_ksp_dict = Pickle.load(open('mrt_map/od_ksp_dict.pkl','rb'))

od_ksp_list = od_ksp_dict["54_85"]
pprint(od_ksp_list[0])

[2100.0,
 '54_85_0',
 ['54',
  '55',
  '56',
  '57',
  '58',
  '59',
  '69',
  '2',
  '70',
  '80',
  '94',
  '81',
  '82',
  '83',
  '19',
  '84',
  '85'],
 {'19_84': 120.0,
  '2_70': 180.0,
  '54_55': 60.0,
  '55_56': 180.0,
  '56_57': 120.0,
  '57_58': 120.0,
  '58_59': 180.0,
  '59_69': 120.0,
  '69_2': 120.0,
  '70_80': 120.0,
  '80_94': 120.0,
  '81_82': 120.0,
  '82_83': 180.0,
  '83_19': 120.0,
  '84_85': 120.0,
  '94_81': 120.0}]


In [26]:
# global settings
slice_len=300
start_ticks=16800
end_ticks=90000
n_station=104
min_od_size=10
# for eliminating the small cluster
min_cluster_ratio = 0.05

n_slice = int((end_ticks-start_ticks)/slice_len)

In [9]:
# pr_dict: o,d,t_o,pid --> pr  
pr_dict= dict()
# n_dict: o, t_o --> count
n_dict=dict()
# w_dict: o,d,t_o,pid --> (mean of waiting time, std of waiting time)
w_dict=dict()

# initialize dictionary for all o and d
for i in range(n_station):
    o_i = str(i)
    n_dict[o_i]=dict()
    pr_dict[o_i]=dict()
    w_dict[o_i]=dict()
    for j in range(n_station):
        d_j = str(j)
        pr_dict[o_i][d_j]=dict()
        w_dict[o_i][d_j]=dict()
        for entry_slice in range(n_slice):
            pr_dict[o_i][d_j][entry_slice]=dict()
            w_dict[o_i][d_j][entry_slice]=dict()
            n_dict[o_i][entry_slice]=0.

In [10]:
# Compute P(path=pid, D=d|O=o, T_o_slice = entry_slice)

from smrt_util import find_the_best_fitting_path
from smrt_util import dict_updater
# slice 43~slice 51 are more than 600 "big" od pair
# Step 1: for each time slice
# for i in range(1,n_slice):
for i in range(43,52):
    ttt=time.time()
    # step 2: for each different O_D
    df_i = df[df.depart_slice==i]
    ods = df_i['od'].unique()
    t = time.time()
    for od in ods:
        # step 3: for each od... (around 37sec all df_od has totally 600 records)
        o, d = od.split("_")
        df_od = df_i[(df_i.od==od)]
        if len(df_od)>=min_od_size:
            # step 4: do gmm
            V = df_od[['duration','entry_slice']].copy()
            X = np.transpose(np.array([V['duration']]))
            best_gmm, best_cv = gmm_selection(X,5)
            clusters =  gmm_clustering(X,best_cv,best_gmm.n_components)
            V['cid'] = clusters
            
            # step 5: eliminate small clusters    
            delta = min_cluster_ratio * len(df_od) 
            S = V.groupby('cid').size()
            good_cids = S[S>delta].keys().tolist()
            
            # step 6: for each cluster, find the best fitting path
            for cid in good_cids:
                good_df = V[V['cid']==cid]
                # the best fitting pack: (cost, path, dict), lucky_duration
                best_fitting_pack, lucky_duration = find_the_best_fitting_path(good_df,od,od_ksp_dict)

                # unpack it                
                best_cost = best_fitting_pack[0]
                pid = best_fitting_pack[1]
                best_path = best_fitting_pack[2]
                best_dict = best_fitting_pack[3]
                
                for _, row in good_df.iterrows():
                    # compute three dicts
                    entry_slice=row.entry_slice
                    #[tech_debt] sometimes the walking_time is negative --> expected time is larger than actual time 
                    walking_time = max(0,(lucky_duration-best_cost)/2)
                    # update dictionaries                    
                    pr_dict,n_dict,w_dict = dict_updater(pr_dict,n_dict,w_dict,o,d,entry_slice,pid,walking_time)
    print("[Info] Slice #"+str(i)+" done.")
    print("       Time:",time.time()-ttt)
    print("       Slice size:",len(df_i))
print("** All Done! **")

[Info] Slice #43 done.
       Time: 43.14327263832092
       Slice size: 22690
[Info] Slice #44 done.
       Time: 47.92849659919739
       Slice size: 24395
[Info] Slice #45 done.
       Time: 44.64749479293823
       Slice size: 23813
[Info] Slice #46 done.
       Time: 43.366055965423584
       Slice size: 23991
[Info] Slice #47 done.
       Time: 45.98386359214783
       Slice size: 24363
[Info] Slice #48 done.
       Time: 47.492878675460815
       Slice size: 25592
[Info] Slice #49 done.
       Time: 47.48857092857361
       Slice size: 25349
[Info] Slice #50 done.
       Time: 47.82138919830322
       Slice size: 26112
[Info] Slice #51 done.
       Time: 41.3821485042572
       Slice size: 23012
** All Done! **


In [ ]:
import collections

# derive the final pr_dict
for o in n_dict:
    for entry_slice in n_dict[o]:
        for d in pr_dict[o]:
            if entry_slice in pr_dict[o][d]:
                for pid in pr_dict[o][d][entry_slice]:
                    pr = pr_dict[o][d][entry_slice][pid]
                    n = n_dict[o][entry_slice]
                    w = w_dict[o][d][entry_slice][pid]
                    counter=collections.Counter(w)
                    for walking_time in counter:
                        counter[walking_time]=counter[walking_time]/pr
                    w_dict[o][d][entry_slice][pid]=counter
                    pr_dict[o][d][entry_slice][pid]=pr/n    

In [4]:
w_dict["23"]["0"][42]

{'23_0_3': [94.5,
  94.5,
  94.5,
  94.5,
  69.509999999999991,
  69.509999999999991,
  69.509999999999991,
  69.509999999999991,
  69.509999999999991,
  69.509999999999991,
  69.509999999999991,
  69.509999999999991,
  69.509999999999991,
  69.509999999999991]}

In [5]:
import collections

# derive the final pr_dict
for o in n_dict:
    for entry_slice in n_dict[o]:
        for d in pr_dict[o]:
            if entry_slice in pr_dict[o][d]:
                for pid in pr_dict[o][d][entry_slice]:
                    pr = pr_dict[o][d][entry_slice][pid]
                    n = n_dict[o][entry_slice]
                    w = w_dict[o][d][entry_slice][pid]
                    counter=collections.Counter(w)
                    for walking_time in counter:
                        counter[walking_time]=counter[walking_time]/pr
                    w_dict[o][d][entry_slice][pid]=counter
                    pr_dict[o][d][entry_slice][pid]=pr/n            

In [6]:
# just for backup
Pickle.dump(pr_dict,open('temp_pr_dict.pkl','wb'))
Pickle.dump(w_dict,open('temp_w_dict.pkl','wb'))
Pickle.dump(n_dict,open('temp_n_dict.pkl','wb'))

# load the backup dicts
pr_dict = Pickle.load(open('temp_pr_dict.pkl','rb'))
w_dict = Pickle.load(open('temp_w_dict.pkl','rb'))
n_dict = Pickle.load(open('temp_n_dict.pkl','rb'))

In [41]:
def predict(o,date,entry_info,delta_t,pr_dict=pr_dict,w_dict=w_dict,od_ksp_dict=od_ksp_dict,info_mode="time"):
    """
    o: starting station (starting station)
    date: yyyy/mm/dd (entry date)
    t_o: hh:mm:ss (entry time)
    delta: seconds (time passed)
    """
    entry_ticks=0
    if info_mode=="time":
        hh,mm,ss = entry_info.split(":")    
        entry_ticks = int(hh)*3600 + int(mm)*60 + int(ss)
    elif info_mode=="ticks":
        entry_ticks = int(entry_info)
    else:
        print("Mode Error")
        return {}  
    
    entry_slice = ticks_to_slice(entry_ticks)
    #print("the entry slice is :",entry_slice)
    
    dest_pr=dict()
    # get all possible pathes and probabilities from o to d starting from t_o_slice
    for d in pr_dict[o]:
        if pr_dict[o][d][entry_slice]!={}:
            for pid in pr_dict[o][d][entry_slice]:
                _,_,p = pid.split("_")
                # get the probability of pid
                pr = pr_dict[o][d][entry_slice][pid]
                path_time,_,path_list,path_dict = od_ksp_list = od_ksp_dict[o+"_"+d][int(p)]
                
                expected_walking_time = 0
                for walking_time in w_dict[o][d][entry_slice][pid]:
                    expected_walking_time += walking_time * w_dict[o][d][entry_slice][pid][walking_time]
                
                # the time starts to travel
                left_t = delta_t - expected_walking_time
                
                # derive which station users are in
                current_station = path_list[0]
                for i in range(len(path_list)-1):
                    seg_id = path_list[i]+"_"+path_list[i+1]
                    seg_time = path_dict[seg_id]
                    left_t-=seg_time
                    #print(seg_id,left_t)
                    if delta_t<0:
                        current_station=path_list[i]
                        break
                
                if left_t - expected_walking_time < 0:
                    # in the last station
                    current_station=path_list[i]
                else:
                    # already outside the station
                    current_station=str(n_station)
                
                dest_pr[current_station]=pr
    return dest_pr

In [45]:
predict("23","2017/03/12","08:02:03",600)

{'1': 0.024390243902439025,
 '104': 0.054878048780487805,
 '11': 0.018292682926829267,
 '13': 0.09146341463414634,
 '14': 0.12195121951219512,
 '15': 0.03048780487804878,
 '17': 0.024390243902439025,
 '19': 0.06097560975609756,
 '22': 0.12195121951219512,
 '30': 0.04878048780487805,
 '33': 0.024390243902439025,
 '45': 0.13414634146341464,
 '50': 0.06097560975609756,
 '52': 0.14634146341463414,
 '84': 0.03048780487804878}